In [1]:
#using DifferentialEquations
using Plots, Colors, LaTeXStrings
using Plots.Measures
font_title = Plots.font("Arial", 24)
font_axis = Plots.font("Arial", 12)
font_legend = Plots.font("Arial", 8)
pyplot(titlefont=font_title, guidefont = font_axis, legendfont = font_legend)

#Define different symbols used in the plotting
delta = latexstring("\$\\delta\$")
micro = latexstring("\$\\mu\$")
vt = latexstring("\$V_t\$")
nt = latexstring("\$N_t\$")
bt = latexstring("\$B_t\$")
wt = latexstring("\$W_t\$")
i_ach = latexstring("\$I_{ACh}\$")

v_color = :deepskyblue
n_color = :magenta
c_color = :green
a_color = :purple
b_color = :red
e_color = :blue
w_color = :gray

figure_path = "C:/users/mtarc/JuliaScripts/RetinalChaos/Notebooks/Figures"

┌ Info: Precompiling Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]
└ @ Base loading.jl:1260
┌ Info: Precompiling PyPlot [d330b81b-6aea-500a-939a-2ce795aea3ee]
└ @ Base loading.jl:1260


"C:/users/mtarc/JuliaScripts/RetinalChaos/Notebooks/Figures"

In [2]:
#If you have a GPU available, you can run the code using 
#using CuArrays
#gpu = true; CuArrays.allowscalar(false)

In [3]:
#otherwise set gpu to false
gpu = false;

In [4]:
using Dates, Plots             
using Logging, TerminalLoggers
global_logger(TerminalLogger());

In [5]:
import RetinalChaos: Network, extract_dict, read_JSON
import RetinalChaos: tar_conds, tar_pars

[ Info: Precompiling RetinalChaos [3f100aef-eb91-46ae-997e-7902479fb568]


In [6]:
dt = 10.0; tspan = (0.0, 300e3)
nx = 96; ny = 96;

In [7]:
SACnet = Network(nx, ny; μ = 0.65, gpu = gpu, version = :gACh)
p_dict = read_JSON("params.json");
u_dict = read_JSON("conds.json");
u0 = extract_dict(u_dict, tar_conds, (nx, ny));
p0 = extract_dict(p_dict, tar_pars);

In [8]:
import RetinalChaos: SDEProblem, noise_2D, SOSRI, solve

In [9]:
println("Warming up solution")
prob = SDEProblem(SACnet, noise_2D, u0, (0.0, 60e3), p0);
@time sol = solve(
    prob,
    SOSRI(),
    abstol = 0.2,
    reltol = 2e-2,
    maxiters = 1e7,
    progress = true, 
    save_everystep = false
)

Warming up solution


SDE   0%|█                                              |  ETA: N/A
SDE   0%|█                                              |  ETA: 0:55:39
SDE   1%|█                                              |  ETA: 1:07:43
SDE   1%|█                                              |  ETA: 1:20:32
SDE   1%|█                                              |  ETA: 1:29:15
SDE   1%|█                                              |  ETA: 1:36:22
SDE   1%|█                                              |  ETA: 1:43:23
SDE   1%|█                                              |  ETA: 1:47:01
SDE   1%|█                                              |  ETA: 1:50:31
SDE   1%|█                                              |  ETA: 1:52:18
SDE   1%|█                                              |  ETA: 1:54:03
SDE   2%|█                                              |  ETA: 1:54:53
SDE   2%|█                                              |  ETA: 1:55:09
SDE   2%|█                                              |  ETA: 1:53

969.032459 seconds (113.52 M allocations: 11.686 GiB, 0.29% gc time)



SDE 100%|███████████████████████████████████████████████| Time: 0:15:44


retcode: Success
Interpolation: 1st order linear
t: 2-element Array{Float64,1}:
     0.0
 60000.0
u: 2-element Array{Array{Float64,3},1}:
 [-65.0 -65.0 … -65.0 -65.0; -65.0 -65.0 … -65.0 -65.0; … ; -65.0 -65.0 … -65.0 -65.0; -65.0 -65.0 … -65.0 -65.0]

[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

[0.085 0.085 … 0.085 0.085; 0.085 0.085 … 0.085 0.085; … ; 0.085 0.085 … 0.085 0.085; 0.085 0.085 … 0.085 0.085]

[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]
 [-73.74009110243092 -66.03875635758601 … -67.35918595327938 -66.36504852263626; -70.7298537943308 -70.53964960296398 … -65.2567943590115 -67.83916498218022; … ; -77.53234559096109 -76.07359552048042 … -71.38911400907845 -73

In [10]:
prob = SDEProblem(prob.f, prob.g, sol[end], (0.0, 60e3), prob.p);
sol = solve(
        prob,
        SOSRI(),
        abstol = 0.2,
        reltol = 2e-2,
        maxiters = 1e7,
        progress = true, 
        saveat = dt,
    )

SDE   0%|█                                              |  ETA: N/A
SDE   2%|█                                              |  ETA: 0:09:12
SDE   4%|██                                             |  ETA: 0:08:08
SDE   5%|███                                            |  ETA: 0:08:18
SDE   6%|███                                            |  ETA: 0:08:59
SDE   6%|███                                            |  ETA: 0:09:48
SDE   7%|████                                           |  ETA: 0:10:42
SDE   7%|████                                           |  ETA: 0:11:30
SDE   8%|████                                           |  ETA: 0:12:19
SDE   8%|████                                           |  ETA: 0:13:01
SDE   8%|████                                           |  ETA: 0:13:39
SDE   9%|█████                                          |  ETA: 0:14:18
SDE   9%|█████                                          |  ETA: 0:14:53
SDE  10%|█████                                          |  ETA: 0:15

retcode: Success
Interpolation: 1st order linear
t: 6001-element Array{Float64,1}:
     0.0
    10.0
    20.0
    30.0
    40.0
    50.0
    60.0
    70.0
    80.0
    90.0
   100.0
   110.0
   120.0
     ⋮
 59890.0
 59900.0
 59910.0
 59920.0
 59930.0
 59940.0
 59950.0
 59960.0
 59970.0
 59980.0
 59990.0
 60000.0
u: 6001-element Array{Array{Float64,3},1}:
 [-73.74009110243092 -66.03875635758601 … -67.35918595327938 -66.36504852263626; -70.7298537943308 -70.53964960296398 … -65.2567943590115 -67.83916498218022; … ; -77.53234559096109 -76.07359552048042 … -71.38911400907845 -73.77387720712788; -77.21682080898638 -68.09316414483678 … -73.78387752196046 -74.6290427217869]

[8.954471017799548e-7 8.094105422393357e-6 … 5.562758190232179e-6 7.352327394310816e-6; 2.1148017043820537e-6 2.2344176907850388e-6 … 1.0103080044927248e-5 4.82833735969606e-6; … ; 3.03244453333803e-7 4.5950340553165386e-7 … 1.7536492670744442e-6 8.85925607701481e-7; 3.3163908283505927e-7 4.496584913464354e-6 … 8.8418485

### [1.5.b] Graphing and visualization

In [11]:
import RetinalChaos: calculate_threshold, get_timestamps, max_interval_algorithim, extract_burstmap, timescale_analysis

In [12]:
vt_sol = Array(sol[:,:,1,:]);
nt_sol = Array(sol[:,:,2,:]);
ct_sol = Array(sol[:,:,3,:]);
at_sol = Array(sol[:,:,4,:]);
bt_sol = Array(sol[:,:,5,:]);
et_sol = Array(sol[:,:,6,:]);

In [27]:
maximum(et_sol)

2.8618036709103927

In [28]:
thresh = calculate_threshold(vt_sol)
spike_array = vt_sol .> thresh;

In [29]:
x_locs = round.(Int, LinRange(1, nx, 10));
y_locs = round.(Int, LinRange(1, ny, 10));

In [ ]:
burst_map = extract_burstmap(spike_array);

@time anim = @animate for i = 1:10:size(sol, 4)
    println("Animating frame $i")
    
    p1 = plot(layout = grid(2, 2), size = (1000,1000))
    heatmap!(p1[1], vt_sol[:, :, i], ratio = :equal, grid = false,
        xaxis = "", yaxis = "", xlims = (0, nx), ylims = (0, ny),
        c = :curl, clims = (-70.0, 0.0),
    )
    scatter!(p1[1], x_locs, y_locs, marker = :star, c = :yellow, label = "")
    heatmap!(p1[2], ct_sol[:, :, i], ratio = :equal, grid = false,
        xaxis = "", yaxis = "", xlims = (0, nx), ylims = (0, ny),
        c = :kgy, clims = (0.0, 1.0),
    )
    scatter!(p1[2], x_locs, y_locs, marker = :star, c = :yellow, label = "")
    heatmap!(p1[3], et_sol[:, :, i], ratio = :equal, grid = false,
        xaxis = "", yaxis = "", xlims = (0, nx), ylims = (0, ny),
        c = :bgy, clims = (0.0, 3.0),
    )
    scatter!(p1[3], x_locs, y_locs, marker = :star, c = :yellow, label = "")
    heatmap!(p1[4], bt_sol[:, :, i], ratio = :equal, grid = false,
        xaxis = "", yaxis = "", xlims = (0, nx), ylims = (0, ny),
        c = :reds, clims = (0.0, 1.0),
    )
    scatter!(p1[4], x_locs, y_locs, marker = :star, c = :yellow, label = "")
    
    p2 = plot(vt_sol[x_locs[1], y_locs[1], :], layout = grid(3, 1), label = "", size = (500,150))
    plot!(p2[2], ct_sol[x_locs[1], y_locs[1], :], c = :kgy, line_z = 1, label = "")
    plot!(p2[3], bt_sol[x_locs[1], y_locs[1], :], c = :reds, line_z = 1, label = "")
    for i = 2:length(x_locs)
        plot!(p2[1], vt_sol[x_locs[i], y_locs[i], :], c = :curl, line_z = i, label = "")
        plot!(p2[2], ct_sol[x_locs[i], y_locs[i], :], c = :kgy, line_z = i, label = "")
        plot!(p2[3], bt_sol[x_locs[i], y_locs[i], :], c = :reds, line_z = i, label = "")
    end
    p2
    vline!(p2[1], [i], label = "", lw = 3.0, c = :black)
    vline!(p2[2], [i], label = "", lw = 3.0, c = :black)
    vline!(p2[3], [i], label = "", lw = 3.0, c = :black)
    p = plot(p2, p1, layout = grid(2,1), size = (1000,1000)) 
end

Animating frame 1
Animating frame 11
Animating frame 21
Animating frame 31
Animating frame 41
Animating frame 51
Animating frame 61
Animating frame 71
Animating frame 81
Animating frame 91
Animating frame 101
Animating frame 111
Animating frame 121
Animating frame 131
Animating frame 141
Animating frame 151
Animating frame 161
Animating frame 171
Animating frame 181
Animating frame 191
Animating frame 201
Animating frame 211
Animating frame 221
Animating frame 231
Animating frame 241
Animating frame 251
Animating frame 261
Animating frame 271
Animating frame 281
Animating frame 291
Animating frame 301
Animating frame 311
Animating frame 321
Animating frame 331
Animating frame 341
Animating frame 351
Animating frame 361
Animating frame 371
Animating frame 381
Animating frame 391
Animating frame 401
Animating frame 411
Animating frame 421
Animating frame 431
Animating frame 441
Animating frame 451
Animating frame 461
Animating frame 471
Animating frame 481
Animating frame 491
Animating f

In [17]:
mp4(anim, "$(figure_path)/wave_propagation.mp4", fps = 10)
gif(anim, "$(figure_path)/wave_propagation.gif", fps = 10)

UndefVarError: UndefVarError: anim not defined

### [1.5.c] Analysis of the wave Data
- The output of the wave we get is very similar to the output of the trace simulation, except it is in 3D (x, y, d(Var)) with respect to time (t)
- Dispatches of count_interval are available to get all the intervals within the grid. 
    - This function however discards information about the x and y location and adds all the intervals to a single list
- Dispatches of get_timestamps are available max_interval_algorithim
    - These return a tuple with (x, y, data) 
    - for get_timestamps, data is the timestamps
    - for max_interval_algorithgim data is the 
        - 1) Burst Timestamps
        - 2) Duration list
        - 3) Spike per burst list
- A dispatch of the timescale analysis is also available, much like the count intervals all spatial information is lost in the process of conducting the timescale analysis. 
    - In order to conducte more precise spike duration analysis a higher resolution needs to be used which can end up very memory consuming. 

In [18]:
#get_timestamps() returns (x, y, timestamps)
timestamp_data = @time get_timestamps(spike_array);

UndefVarError: UndefVarError: spike_array not defined

In [19]:
#max_interval_algorithim() returns (x, y, [burst_timestamps, durations, spikes_per_burst and intervurst interval])
burst_data = max_interval_algorithim(spike_array);

UndefVarError: UndefVarError: spike_array not defined

In [20]:
#This function breaks downt the spike array into bursts according to the max interval algorithim
burst_map = extract_burstmap(spike_array);

UndefVarError: UndefVarError: spike_array not defined

In [21]:
using Statistics, StatsBase

In [22]:
ts_lattice = timescale_analysis(vm; verbose = 1, mode = 2);

UndefVarError: UndefVarError: vm not defined

#### Comparing to 1D traces

In [23]:
import RetinalChaos: T_sde

In [24]:
p = read_JSON("params.json") |> extract_dict;
u0 = read_JSON("conds.json") |> extract_dict;
dt = 1.0
tspan = (0.0, 500e3);

In [25]:
SDEprob = SDEProblem(T_sde, u0, tspan, p)
println("Time it took to simulate 60s:")
@time SDEsol = solve(SDEprob, SOSRI(), abstol = 2e-2, reltol = 2e-2, maxiters = 1e7, saveat = dt); 
trace = Array(SDEsol)';

Time it took to simulate 60s:
  6.839126 seconds (25.04 M allocations: 1.152 GiB, 9.08% gc time)


In [26]:
vt_iso = trace[:, 1];

In [ ]:
#Try to get these solutions to line up by adjusting this value
iso_begin = 265000
xlims = (0.0, 60e3) 
xticks = (collect(xlims[1]:5000:xlims[2]), collect(0:5.0:xlims[2]/1000))

fig5_A = plot(vt_iso[iso_begin:Int(iso_begin+60e3)],  label = "Isolated SAC", 
    lw = 2.0, c = v_color
)
plot!(fig5_A, sol.t, vt_sol[21,10,:], label = "SAC in Network", 
    lw = 2.0, c = e_color, linestyle = :dash, 
    xticks = xticks 
)

In [ ]:
#Conducting the analysis in a single function you can use the imported function timescale_analysis
ts_iso = timescale_analysis(trace[:,1]; dt = dt, verbose = 1, mode = 2);

In [ ]:
bins = LinRange(minimum(ts_lattice[2]), maximum(ts_lattice[2]), 100);
fig5_B = histogram(ts_iso[2], normalize = :pdf, xlims = (500,1500), c = v_color, ylabel = "Probability of Burst");
fig5_B = histogram!(ts_lattice[2], normalize = :pdf, xlims = (500,1500), c = e_color, xlabel = "Burst Length (ms)")
#fig5_B = plot(fig5_Ba, fig5_Bb)

In [ ]:
fig5 = plot(fig5_A, fig5_B, layout = grid(2,1))

### [1.5.d] We can do multiple runs of this simulation loop. This allows us to 
- Perform repeated trials
- Alter a variable

I will only be doing one of these however, because these trials are costly. The below code should be performed only on a computer with a capable GPU, otherwise this will take a very long time. I will be altering the amount of acetylcholine released by a single SAC in $\mu M / ms$($\rho$)

In [ ]:
par = :ρ
n_trials = 10
par_range = LinRange(1.0, 10.0, n_trials)

In [ ]:
for val in par_range
    with_logger(TerminalLogger()) do
        #Warmup
        SDE_mat_prob = SDEProblem(SACnet, noise_2D, u0, (0.0, 60e3), p0);
        SDE_mat_sol = solve(
            SDE_mat_prob,
            SOSRI(),
            abstol = 0.2,
            reltol = 2e-2,
            maxiters = 1e7,
            progress = true, 
            saveat = dt,
        )
        #Run the trial
        u0_new = SDE_mat_sol[end]
        SDE_mat_prob = SDEProblem(SACnet, noise_2D, u0_new, tspan, p0);
        SDE_mat_sol = solve(
            SDE_mat_prob,
            SOSRI(),
            abstol = 0.2,
            reltol = 2e-2,
            maxiters = 1e7,
            progress = true, 
            saveat = dt,
        )
        #Perform the Data Analysis TODO
    end
end